#### 

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
df = pd.read_csv('municipality_bus_utilization.csv', index_col=0)  
df

---
#### First we go through steps for municipality 0

In [3]:
import warnings
warnings.filterwarnings('ignore')

df2 = df[df['municipality_id'] == 1]
df2.index = pd.to_datetime(df2.index)
df2["time"] = pd.to_datetime(df2.index)

df3 = df2.resample('60min', offset=0, label='right')['usage'].max()
df3 = df3.to_frame()
df3 = df3.between_time('8:00', '17:00')

for i in range(len(df3)):
    if(df3.index[i].hour==8 and pd.isna(df3['usage'][i])):
        df3.iloc[i]['usage'] = df3.iloc[i+1]['usage']
        
df3['usage'] = df3['usage'].interpolate()
df3

,usage
timestamp,
2017-06-04 08:00:00,129.0
2017-06-04 09:00:00,164.0
2017-06-04 10:00:00,245.0
2017-06-04 11:00:00,317.0
2017-06-04 12:00:00,341.0
...,...
2017-08-19 13:00:00,406.0
2017-08-19 14:00:00,408.0
2017-08-19 15:00:00,410.0


In [17]:
df_train1 = df3[:-140]
df_test1  = df3[-140:-70]
num_week1 = len(df_train1)/70
print('Number of week for first week training is : {}'.format(week_num1))

df_train2 = df3[:-70]
df_test2  = df3[-70:]
num_week2 = len(df_train2)/70
print('Number of week for second week training is : {}'.format(week_num2))

df_train2

Number of week for first week training is : 9.0
Number of week for second week training is : 10.0


,usage
timestamp,
2017-06-04 08:00:00,129.0
2017-06-04 09:00:00,164.0
2017-06-04 10:00:00,245.0
2017-06-04 11:00:00,317.0
2017-06-04 12:00:00,341.0
...,...
2017-08-12 13:00:00,406.0
2017-08-12 14:00:00,407.0
2017-08-12 15:00:00,411.0


In [18]:
from sklearn.metrics import mean_squared_error
from math import sqrt


predict_week1 = np.zeros(70)
for i in range(70):
    tmp = 0
    for j in range(int(num_week1)):
       tmp += df_train1['usage'][70*j+i]
    predict_week1[i] = tmp/num_week1
rmse1 = sqrt(mean_squared_error(np.array(df_test1['usage']), predict_week1))
print('Test RMSE first week: %.3f' % rmse1)

predict_week2 = np.zeros(70)
for i in range(70):
    tmp = 0
    for j in range(int(num_week2)):
       tmp += df_train2['usage'][70*j+i]
    predict_week2[i] = tmp/num_week2
rmse1 = sqrt(mean_squared_error(np.array(df_test2['usage']), predict_week2))
print('Test RMSE second week: %.3f' % rmse1)


Test RMSE first week: 72.529
Test RMSE second week: 58.000


---
#### No we write a function to get number of municipality and return prediction of week and print RMSE

In [24]:
def predict_test(manip_id):
    
    df2 = df[df['municipality_id'] == manip_id]
    df2.index = pd.to_datetime(df2.index)
    df2["time"] = pd.to_datetime(df2.index)

    df3 = df2.resample('60min', offset=0, label='right')['usage'].max()
    df3 = df3.to_frame()
    df3 = df3.between_time('8:00', '17:00')

    for i in range(len(df3)):
        if(df3.index[i].hour==8 and pd.isna(df3['usage'][i])):
            df3.iloc[i]['usage'] = df3.iloc[i+1]['usage']

    df3['usage'] = df3['usage'].interpolate()
    
    ##=======================================
    df_train1 = df3[:-140]
    df_test1  = df3[-140:-70]
    num_week1 = len(df_train1)/70
    df_train2 = df3[:-70]
    df_test2  = df3[-70:]
    num_week2 = len(df_train2)/70
    ##=======================================
    
    predict_week1 = np.zeros(70)
    for i in range(70):
        tmp = 0
        for j in range(int(num_week1)):
           tmp += df_train1['usage'][70*j+i]
        predict_week1[i] = tmp/num_week1
    rmse1 = sqrt(mean_squared_error(np.array(df_test1['usage']), predict_week1))
    print('Test RMSE first week for municipality {} is: {}'.format(manip_id, rmse1))

    predict_week2 = np.zeros(70)
    for i in range(70):
        tmp = 0
        for j in range(int(num_week2)):
           tmp += df_train2['usage'][70*j+i]
        predict_week2[i] = tmp/num_week2
    rmse1 = sqrt(mean_squared_error(np.array(df_test2['usage']), predict_week2))
    print('Test RMSE second week for municipality {} is: {}'.format(manip_id, rmse1))
    
    
    return predict_week1, predict_week2


In [25]:
for i in range(10):
    predict_week1, predict_week2 = predict_test(i)

Test RMSE first week for municipality 0 is: 89.94138957123033
Test RMSE second week for municipality 0 is: 470.7193005445753
Test RMSE first week for municipality 1 is: 72.52938933759387
Test RMSE second week for municipality 1 is: 57.999704530485175
Test RMSE first week for municipality 2 is: 80.31931659980894
Test RMSE second week for municipality 2 is: 73.37343297026585
Test RMSE first week for municipality 3 is: 222.28981702384266
Test RMSE second week for municipality 3 is: 175.474216198871
Test RMSE first week for municipality 4 is: 541.519634814977
Test RMSE second week for municipality 4 is: 452.8910281460435
Test RMSE first week for municipality 5 is: 68.95258127299319
Test RMSE second week for municipality 5 is: 59.26110907895828
Test RMSE first week for municipality 6 is: 111.98637559959708
Test RMSE second week for municipality 6 is: 441.8479666729962
Test RMSE first week for municipality 7 is: 198.874811496383
Test RMSE second week for municipality 7 is: 155.62564551761602

#### Here we have used simple method of averaging over the same hour of the same day of week over available weeks